In [1]:
import os
import openpyxl
import opyplus as op

# Alireza Habibi - 99201335
# Hamidreza Fathi - 99201357

# idf path
DOEE_path = "C:\\Users\Acer\Desktop\DOEE"
idf_path = os.path.join(DOEE_path,"DOEE3.idf")

In [2]:
epm = op.Epm.load(idf_path)

RP  = epm.RunPeriod
BSD = epm.Buildingsurface_Detailed
FSD = epm.Fenestrationsurface_Detailed

In [3]:
epw_city = ["Tehran.epw","Tabriz.epw","Qum.epw"]
building_dir = [0, 90, 180, 270]
numb_EC = len(epw_city)
numb_BD = len(building_dir)

epm.RunPeriod[0].begin_month = 5
epm.RunPeriod[0].end_month = 8
numb_M = (RP[0].end_month - RP[0].begin_month + 1)

time = [2678400, 2592000, 2678400 , 2678400]

In [4]:
Result_EP = [[[0 for month in range(numb_M)] for degree in range(numb_BD)] for city in range(numb_EC)]
EP = [[[0 for month in range(numb_M)] for degree in range(numb_BD)] for city in range(numb_EC)]
Result_St = [[[0 for month in range(numb_M)] for degree in range(numb_BD)] for city in range(numb_EC)]
St = [[[0 for month in range(numb_M)] for degree in range(numb_BD)] for city in range(numb_EC)]

In [5]:
BSD_name  = []; FSD_name    = []
area_wall = []; area_window = []

H_D = 0; 
area_subs = 0
U_window  = 4
for i in range (0,len(FSD)):
    if (FSD[i].surface_type == "window"):
        FSD_name.append(FSD[i].building_surface_name[0])
        x1 = FSD[i].vertex_1_x_coordinate; y1 = FSD[i].vertex_1_y_coordinate; z1 = FSD[i].vertex_1_z_coordinate
        x2 = FSD[i].vertex_2_x_coordinate; y2 = FSD[i].vertex_2_y_coordinate; z2 = FSD[i].vertex_2_z_coordinate
        x3 = FSD[i].vertex_3_x_coordinate; y3 = FSD[i].vertex_3_y_coordinate; z3 = FSD[i].vertex_3_z_coordinate
        aw = ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)**(0.5)
        bw = ((x3 - x2)**2 + (y3 - y2)**2 + (z3 - z2)**2)**(0.5)
        area_subs = aw*bw
        area_window.append(area_subs)
        H_D += U_window*(area_subs)

area_sur = 0
U_wall   = 1.11
for j in range(0,len(BSD)):
    if (BSD[j].outside_boundary_condition == "outdoors" and BSD[j].surface_type == "wall"):
        x1 = BSD[j].vertex_1_x_coordinate; y1 = BSD[j].vertex_1_y_coordinate; z1 = BSD[j].vertex_1_z_coordinate
        x2 = BSD[j].vertex_2_x_coordinate; y2 = BSD[j].vertex_2_y_coordinate; z2 = BSD[j].vertex_2_z_coordinate
        x3 = BSD[j].vertex_3_x_coordinate; y3 = BSD[j].vertex_3_y_coordinate; z3 = BSD[j].vertex_3_z_coordinate
        a = ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)**(0.5)
        b = ((x3 - x2)**2 + (y3 - y2)**2 + (z3 - z2)**2)**(0.5)
        if (BSD[j].name in FSD_name):
            ind = FSD_name.index(BSD[j].name)
            area_sur = a*b - area_window[ind]
        else:
            area_sur = a*b
        H_D += U_wall*area_sur
        BSD_name.append(BSD[j].name)
        area_wall.append(area_sur)

U_roof = 1.46
area_roof = 1314
H_D += U_roof*area_roof

H_G = 0
U_ground = 1
area_ground = 1314
H_G = 1.5*U_ground*area_ground

H_tr = H_D + H_G

In [6]:
for i in range(0,numb_EC):
    
    # epw path
    epw_path = os.path.join(DOEE_path,"WeatherData",epw_city[i])
    
    for j in range(0,numb_BD):
        
        # change direction of biulding
        epm.Building[0].north_axis = building_dir[j]
        
        # run simulation
        s = op.simulate(epm, epw_path, "Results")
        
        # retrieve hourly output (.eso file)
        monthly_output = s.get_out_eso()

        # ask for datetime index on year 2006
        monthly_output.create_datetime_index(2006)

        # get Pandas dataframe
        df = monthly_output.get_data()
        
        dfc = df.columns
        numb_dfc = len(dfc)
        
        Result_EP[i][j] = df["doee - wholebuilding,Zone List Sensible Cooling Energy"]
        
        
        Ti  = [0 for month in range(numb_M)]
        numb_Ti = 0
        q_ve  = [0 for month in range(numb_M)]
        q_in  = [0 for month in range(numb_M)]
        PhiPeaple = [0 for month in range(numb_M)]
        PhiLight  = [0 for month in range(numb_M)]
        PhiApp    = [0 for month in range(numb_M)]
        SolPhiG = [0 for month in range(numb_M)]
        SolPhiO = [0 for month in range(numb_M)]
        IsolO   = [0 for month in range(numb_M)]

        Rse_wall   = 0.04; Rse_roof   = 0.04
        Uc_wall    = 1.11; Uc_roof    = 1.46
        alpha_wall = 0.7;  alpha_roof = 0.7
        Fr_wall    = 0.5;  Fr_roof    = 1
        Fsh = 1
        hr = 5*0.75
        deltaTer = 11
        
        for k in range(0, numb_dfc):
            if ("Mean Air Temperature" in dfc[k]):
                Ti += df[dfc[k]]
                numb_Ti +=1
            if ("Ventilation" in dfc[k]):
                q_ve += df[dfc[k]]
            if ("Infiltration" in dfc[k]):
                q_in += df[dfc[k]]
            if ("People" in dfc[k]):
                PhiPeaple += df[dfc[k]]
            if ("Lights" in dfc[k]):
                PhiLight += df[dfc[k]]
            # if ("Equipment" in dfc[k]):
            #     PhiApp += df[dfc[k]]
            if ("Windows Total Transmitted" in dfc[k]):
                SolPhiG += df[dfc[k]]
            if (dfc[k][0:11] in BSD_name):
                ind = BSD_name.index(dfc[k][0:11])
                Ac = area_wall[ind]
                Asol = alpha_wall*Rse_wall*Uc_wall*Ac
                Phir = Rse_wall*Uc_wall*hr*Ac*deltaTer
                IsolO = df[dfc[k]]
                IsolO = [Fsh*Asol*x - Fr_wall*Phir for x in IsolO]
                for z in range(0, len(SolPhiO)):
                    SolPhiO[z] += IsolO[z]
            elif (dfc[k][0:11] == "surface 423" or dfc[k][0:11] == "surface 258"):
                if (dfc[k][0:11] == "surface 423"):
                    Ac = 252
                else:
                    Ac = 1052
                Asol = alpha_roof*Rse_roof*Uc_roof*Ac
                Phir = Rse_roof*Uc_roof*hr*Ac*deltaTer
                IsolO = df[dfc[k]]
                IsolO = [Fsh*Asol*x - Fr_roof*Phir for x in IsolO]
                for z in range(0, len(SolPhiO)):
                    SolPhiO[z] += IsolO[z] 
    
    
        Te = df["environment,Site Outdoor Air Drybulb Temperature"]
        Ti = [(x)/numb_Ti for x in Ti]
        deltaT = []
        for num1, num2 in zip(Ti, Te):
            deltaT.append(num1 - num2)
        TT = []
        for num1, num2 in zip(deltaT, time):
            TT.append(num1 * num2)
            
            
        Q_tr = [x*H_tr for x in TT]
        
        
        q_ve_t = []
        for num1, num2 in zip(q_ve, q_in):
            q_ve_t.append(num1 + num2)
        f_ve = [1, 1, 1, 1]
        H_ve = []
        for num1, num2 in zip(f_ve, q_ve_t):
            H_ve.append(num1 * num2)
        H_ve = [1200*x for x in H_ve]
        Q_ve = []
        for num1, num2 in zip(TT, H_ve):
            Q_ve.append(num1 * num2)
           
        
        IntPhi = []
        for num1, num2, num3 in zip(PhiPeaple, PhiLight, PhiApp):
            IntPhi.append(num1 + num2 + num3)
        Q_int = []
        for num1, num2 in zip(time, IntPhi):
            Q_int.append(num1 * num2)
            
    
        SolPhi = []
        for num1, num2 in zip(SolPhiG, SolPhiO):
            SolPhi.append(num1 + num2)
        Q_sol = []
        for num1, num2 in zip(time, SolPhi):
            Q_sol.append(num1 * num2)

            
        Q_ht = []
        for num1, num2 in zip(Q_tr, Q_ve):
            Q_ht.append(num1 + num2)            
        Q_gn = []
        for num1, num2 in zip(Q_int, Q_sol):
            Q_gn.append(num1 + num2)

            
        etha = [0 for month in range(numb_M)]
        cm = 370000.0*area_ground
        ta = [(cm/3600.0)/(H_tr + x) for x in H_ve]
        ac0 = 1.0
        ta0 = 15
        ac = [ac0 + x/ta0 for x in ta]
        gama = []
        for num1, num2 in zip(Q_gn, Q_ht):
            gama.append(num1/num2)
        for k in range(0, len(gama)):
            etha[k] = ((1 - 1/abs(gama[k])**(ac[k]))/(1 - 1/abs(gama[k])**(ac[k]+1)))
            
            
        Q_c = []
        for num1, num2, num3 in zip(Q_gn, etha, Q_ht):
            Q_c.append((num1 - num2*num3))
            
        Result_St[i][j] = Q_c
        
        print(epw_city[i], "direction: ", building_dir[j])
        print("          may","             june","          july","           agust")
        print("Ti:    ", ["{:.10f}".format(x) for x in Ti])
        print("Q_tr:  ", ["{:e}".format(x/(3.6*10**6)) for x in Q_tr])
        print("Q_ve:  ", ["{:e}".format(x/(3.6*10**6)) for x in Q_ve])
        print("Q_int: ", ["{:e}".format(x/(3.6*10**6)) for x in Q_int])
        print("Q_sol: ", ["{:e}".format(x/(3.6*10**6)) for x in Q_sol])
        print("etha:  ", ["{:.10f}".format(x) for x in etha])
        print("Q_gn:  ", ["{:e}".format(x/(3.6*10**6)) for x in Q_gn])
        print("Q_ht:  ", ["{:e}".format(x/(3.6*10**6)) for x in Q_ht])
        print("Q_c:   ", ["{:e}".format(x/(3.6*10**6)) for x in Q_c])
        print("\n")
        


called Simulation.from_input on a simulation directory that is not empty (Results)
called Simulation.from_input on a simulation directory that is not empty (Results)


Tehran.epw direction:  0
          may              june           july            agust
Ti:     ['23.9691796359', '27.2611575764', '27.9982002556', '28.1441028773']
Q_tr:   ['2.030149e+04', '-8.748271e+03', '-1.704199e+04', '-1.754246e+04']
Q_ve:   ['2.573483e+04', '-1.345473e+04', '-2.554033e+04', '-2.565803e+04']
Q_int:  ['2.845319e+04', '2.728445e+04', '2.650879e+04', '2.845319e+04']
Q_sol:  ['6.668250e+04', '6.847748e+04', '6.703001e+04', '6.731778e+04']
etha:   ['0.7579055841', '0.8791960698', '0.7646437117', '0.7675558917']
Q_gn:   ['9.513568e+04', '9.576193e+04', '9.353880e+04', '9.577096e+04']
Q_ht:   ['4.603631e+04', '-2.220300e+04', '-4.258232e+04', '-4.320049e+04']
Q_c:    ['6.024450e+04', '1.152827e+05', '1.260991e+05', '1.289298e+05']




called Simulation.from_input on a simulation directory that is not empty (Results)


Tehran.epw direction:  90
          may              june           july            agust
Ti:     ['24.0874788033', '27.4081142007', '28.1091294709', '28.2352800834']
Q_tr:   ['2.137785e+04', '-7.454297e+03', '-1.603268e+04', '-1.671287e+04']
Q_ve:   ['2.720873e+04', '-1.134375e+04', '-2.325828e+04', '-2.385399e+04']
Q_int:  ['2.845319e+04', '2.728445e+04', '2.650879e+04', '2.845319e+04']
Q_sol:  ['7.725117e+04', '8.045398e+04', '7.711866e+04', '7.831812e+04']
etha:   ['0.7689672258', '0.9118708280', '0.8029261369', '0.8031507618']
Q_gn:   ['1.057044e+05', '1.077384e+05', '1.036275e+05', '1.067713e+05']
Q_ht:   ['4.858658e+04', '-1.879805e+04', '-3.929096e+04', '-4.056686e+04']
Q_c:    ['6.834286e+04', '1.248798e+05', '1.351752e+05', '1.393526e+05']




called Simulation.from_input on a simulation directory that is not empty (Results)


...

Qum.epw direction:  0
          may              june           july            agust
Ti:     ['24.9076656770', '27.3860087743', '28.7882149998', '28.3643820882']
Q_tr:   ['-3.976181e+02', '-2.515578e+04', '-4.416176e+04', '-3.749297e+04']
Q_ve:   ['-5.002926e+02', '-3.483694e+04', '-5.991279e+04', '-5.239761e+04']
Q_int:  ['2.845319e+04', '2.728445e+04', '2.650879e+04', '2.845319e+04']
Q_sol:  ['5.118976e+04', '5.294744e+04', '5.284719e+04', '5.325744e+04']
etha:   ['0.9974169518', '0.6472807308', '0.4901371012', '0.5394174910']
Q_gn:   ['7.964295e+04', '8.023189e+04', '7.935598e+04', '8.171063e+04']
Q_ht:   ['-8.979107e+02', '-5.999271e+04', '-1.040745e+05', '-8.989058e+04']
Q_c:    ['8.053854e+04', '1.190640e+05', '1.303668e+05', '1.301992e+05']




called Simulation.from_input on a simulation directory that is not empty (Results)


Qum.epw direction:  90
          may              june           july            agust
Ti:     ['25.0315117523', '27.5220248460', '28.9247572071', '28.4671770559']
Q_tr:   ['7.292132e+02', '-2.395814e+04', '-4.291941e+04', '-3.655768e+04']
Q_ve:   ['9.102338e+02', '-3.230321e+04', '-5.543413e+04', '-4.852743e+04']
Q_int:  ['2.845319e+04', '2.728445e+04', '2.650879e+04', '2.845319e+04']
Q_sol:  ['6.064527e+04', '6.267782e+04', '6.298694e+04', '6.335129e+04']
etha:   ['0.9951179342', '0.6947206409', '0.5422312640', '0.5897775438']
Q_gn:   ['8.909845e+04', '8.996228e+04', '8.949573e+04', '9.180448e+04']
Q_ht:   ['1.639447e+03', '-5.626134e+04', '-9.835355e+04', '-8.508511e+04']
Q_c:    ['8.746701e+04', '1.290482e+05', '1.428261e+05', '1.419858e+05']




called Simulation.from_input on a simulation directory that is not empty (Results)


Qum.epw direction:  180
          may              june           july            agust
Ti:     ['24.9432967403', '27.4790601563', '28.8981814359', '28.4620762463']
Q_tr:   ['-7.342375e+01', '-2.433645e+04', '-4.316122e+04', '-3.660409e+04']
Q_ve:   ['-9.422077e+01', '-3.431252e+04', '-5.961717e+04', '-5.188278e+04']
Q_int:  ['2.845319e+04', '2.728445e+04', '2.650879e+04', '2.845319e+04']
Q_sol:  ['5.166737e+04', '5.332268e+04', '5.327326e+04', '5.321917e+04']
etha:   ['0.9997143666', '0.6538785612', '0.4947169170', '0.5433161635']
Q_gn:   ['8.012056e+04', '8.060714e+04', '7.978205e+04', '8.167236e+04']
Q_ht:   ['-1.676445e+02', '-5.864897e+04', '-1.027784e+05', '-8.848687e+04']
Q_c:    ['8.028816e+04', '1.189564e+05', '1.306283e+05', '1.297487e+05']


Qum.epw direction:  270
          may              june           july            agust
Ti:     ['25.0575767398', '27.5550030137', '28.9536671592', '28.4822357057']
Q_tr:   ['9.663692e+02', '-2.366776e+04', '-4.265637e+04', '-3.642066e+0

In [7]:
for i in range(0,numb_EC):
    for j in range(0,numb_BD):
        EP[i][j] = [(x + 0.0)/(3.6*10**6) for x in Result_EP[i][j]]
        
for i in range(0,numb_EC):
    for j in range(0,numb_BD):    
        St[i][j] = [x/(3.6*10**6) for x in Result_St[i][j]]

In [8]:
for i in range(0,numb_EC):
    print("Energy Plus result in joule for" , epw_city[i], "and direction")
    print("          may","             june","          july","           agust")
    for j in range(0,numb_BD):
        print(building_dir[j], ["{:e}".format(x + 0.0) for x in Result_EP[i][j]])
    print("\n")
        
for i in range(0,numb_EC):
    print("Standard ISO 13790 result in joule for" , epw_city[i], "and direction")
    print("          may","             june","          july","           agust")
    for j in range(0,numb_BD):
        print(building_dir[j], ["{:e}".format(x) for x in Result_St[i][j]])
    print("\n")
        
for i in range(0,numb_EC):
    print("Energy Plus result in kilowatt hour for" , epw_city[i], "and direction")
    print("          may","             june","          july","           agust")
    for j in range(0,numb_BD):
        print(building_dir[j], ["{:e}".format(x) for x in EP[i][j]])
    print("\n")

for i in range(0,numb_EC):
    print("Standard ISO 13790 result in kilowatt hour" , epw_city[i], "and direction")
    print("          may","             june","          july","           agust")
    for j in range(0,numb_BD):
        print(building_dir[j], ["{:e}".format(x) for x in St[i][j]])
    print("\n")

Energy Plus result in joule for Tehran.epw and direction
          may              june           july            agust
0 ['2.033319e+11', '3.247582e+11', '3.615282e+11', '3.827870e+11']
90 ['2.302904e+11', '3.576036e+11', '3.889353e+11', '4.132328e+11']
180 ['2.047132e+11', '3.223834e+11', '3.586101e+11', '3.794348e+11']
270 ['2.270248e+11', '3.582982e+11', '3.875053e+11', '4.049566e+11']


Energy Plus result in joule for Tabriz.epw and direction
          may              june           july            agust
0 ['9.187166e+10', '1.704452e+11', '2.395675e+11', '2.350636e+11']
90 ['9.907709e+10', '1.864338e+11', '2.546815e+11', '2.466391e+11']
180 ['9.267572e+10', '1.688194e+11', '2.351605e+11', '2.313668e+11']
270 ['9.739517e+10', '1.855317e+11', '2.526786e+11', '2.412293e+11']


Energy Plus result in joule for Qum.epw and direction
          may              june           july            agust
0 ['2.325649e+11', '3.348959e+11', '4.142764e+11', '3.990443e+11']
90 ['2.558150e+11', '3.

In [9]:
wb = openpyxl.load_workbook("C:\\Users\Acer\Desktop\DOEE\DOEE-result.xlsx")

for i in range(0,numb_EC):
    for j in range(0,numb_BD):
        for z in range(0,numb_M):
            wb["EP"].cell(row = 3+z, column = 2+j+i*4).value = EP[i][j][z]

for i in range(0,numb_EC):
    for j in range(0,numb_BD):
        for z in range(0,numb_M):
            wb["St"].cell(row = 3+z, column = 2+j+i*4).value = St[i][j][z]

wb.save("C:\\Users\Acer\Desktop\DOEE\DOEE-result.xlsx")